In [ ]:
# run animegan2
# data:Hayao, Paprika, Shinkai, SummerWar

!pip install nvidia-pyindex
!pip install nvidia-tensorflow[horovod]
!conda install -c conda-forge openmpi

!git clone https://github.com/TachibanaYoshino/AnimeGANv2
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/dataset-1/dataset.zip
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/vgg16%2F19.npy/vgg19.npy
!cp -f vgg19.npy AnimeGANv2/vgg19_weight/vgg19.npy
!unzip "dataset.zip" -d "AnimeGANv2/dataset/"

!python train.py --dataset Hayao --epoch 101 --init_epoch 10
!python test.py --checkpoint_dir checkpoint/generator_Hayao_weight --test_dir dataset/test/HR_photo --save_dir Hayao/HR_photo
!tensorboard --bind_all --logdir checkpoint/generator_Hayao_weight

In [65]:
# load animegan vgg19 as torch vgg
# animegan vgg: bgr and 255
import numpy as np
import torch
import torchvision.models as models

vgg19_255 = np.load("vgg19.npy", encoding="latin1", allow_pickle=True).item()
vgg19_255_keys = list(vgg19_255.keys())
vgg19_255_keys.sort()

vgg19 = models.vgg19()
vgg19_state_dict = vgg19.state_dict()

for i, key in enumerate(vgg19_state_dict):
    print(vgg19_255_keys[i // 2], key)
    weight = vgg19_255[vgg19_255_keys[i // 2]]
    weight = weight[0] if "weight" in key else weight[1]

    if "bias" in key:
        pass
    elif "features" in key:
        weight = np.transpose(weight, (3, 2, 0, 1))
    elif "classifier" in key:
        weight = np.transpose(weight, (1, 0))
    vgg19_state_dict[key] = torch.from_numpy(weight)

vgg19.load_state_dict(vgg19_state_dict)

conv1_1 features.0.weight
conv1_1 features.0.bias
conv1_2 features.2.weight
conv1_2 features.2.bias
conv2_1 features.5.weight
conv2_1 features.5.bias
conv2_2 features.7.weight
conv2_2 features.7.bias
conv3_1 features.10.weight
conv3_1 features.10.bias
conv3_2 features.12.weight
conv3_2 features.12.bias
conv3_3 features.14.weight
conv3_3 features.14.bias
conv3_4 features.16.weight
conv3_4 features.16.bias
conv4_1 features.19.weight
conv4_1 features.19.bias
conv4_2 features.21.weight
conv4_2 features.21.bias
conv4_3 features.23.weight
conv4_3 features.23.bias
conv4_4 features.25.weight
conv4_4 features.25.bias
conv5_1 features.28.weight
conv5_1 features.28.bias
conv5_2 features.30.weight
conv5_2 features.30.bias
conv5_3 features.32.weight
conv5_3 features.32.bias
conv5_4 features.34.weight
conv5_4 features.34.bias
fc6 classifier.0.weight
fc6 classifier.0.bias
fc7 classifier.3.weight
fc7 classifier.3.bias
fc8 classifier.6.weight
fc8 classifier.6.bias


<All keys matched successfully>

In [ ]:
# load unet
# learn_gen = unet_learner(dls, resnet34, loss_func=F.l1_loss, blur=True, norm_type=NormType.Weight, self_attention=True, y_range=(-3.0, 3.0))

In [22]:
import torch
from model import Generator

device = torch.device("cpu")
model = Generator().to(device)
model.load_state_dict(torch.load("test.pt"))

<All keys matched successfully>